In [390]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers.embeddings import Embedding
import logging
import numpy as np
import pandas as pd
import math
import numpy as np

In [392]:
#將人工標註結果載入
y_train = []
data_path = r"C:\Users\peihsuan\Downloads\17check_correct\train_text.txt"
df = pd.read_csv(data_path,encoding='big5')  
y_train = df['label']

In [393]:
print(y_train[0:10])

0    0
1    1
2    0
3    1
4    0
5    1
6    1
7    1
8    1
9    0
Name: label, dtype: int64


In [394]:
#將學生答案載入
train_text = []
data_path = r"C:\Users\peihsuan\Downloads\17check_correct\text_embedding.csv"
#df = pd.read_csv(data_path,encoding= 'big5',names=['text','embeddings'])  
df = pd.read_csv(data_path,encoding= 'big5',sep = ';') 
train_text = df['text']

In [395]:
print(train_text[0:10])

0     夾
1    衣服
2    裁縫
3     白
4    民生
5    用品
6    白色
7    白色
8    白色
9     裝
Name: text, dtype: object


In [396]:
#將文字embedding作為其中一項feature儲存
x_train_seq_tmp = []
x_train_seq_tmp = df['embeddings']

In [397]:
#將embedding轉為數字陣列
tmp = []
x_train_seq = []
for items in range(0,len(x_train_seq_tmp)):
    tmp=np.fromstring(x_train_seq_tmp[items], dtype=float, sep=',')
    x_train_seq.append(tmp)
    tmp = []

In [398]:
print(len(train_text))

861


In [399]:
print(len(y_train))

861


In [400]:
print(len(x_train_seq[0]))

250


In [401]:
print((x_train_seq[0]))

[ 1.49791613e-01  2.32222646e-01 -2.35129640e-01  1.40852228e-01
 -1.55842662e-01 -1.11387797e-01  7.35009387e-02  9.97459516e-02
 -3.09015125e-01 -8.17563385e-03  4.08757925e-02  3.01161647e-01
  2.04978168e-01  2.93819653e-03 -8.32686424e-02 -1.80799752e-01
  3.54098916e-01 -5.89631684e-02 -1.31971732e-01 -4.43562157e-02
  2.73908615e-01 -2.13093743e-01  1.71275586e-02  2.80560672e-01
 -1.18280299e-01 -1.15898497e-01  6.96986392e-02  1.04522914e-01
  8.83067995e-02  2.53624246e-02  8.51094946e-02 -9.82208624e-02
 -2.46052817e-02 -1.43778883e-03 -7.31849894e-02 -8.06608200e-02
  3.63679975e-01 -1.07017249e-01 -2.29887813e-01 -4.89401296e-02
  2.62912512e-01  5.20866178e-02  1.92555875e-01  1.98139727e-01
 -2.68179271e-02  5.70187926e-01  9.47752893e-02 -1.48098558e-01
 -2.63165504e-01 -2.65880078e-01  2.17050493e-01 -4.87297028e-02
  2.56990433e-01  1.80252224e-01 -1.28927648e-01  4.10199136e-01
  2.38294885e-01  1.97853059e-01  2.17554703e-01  1.05103571e-02
  1.84963748e-01  4.83086

In [404]:
#將訓練資料切分為訓練及測試
train_size = int(len(train_text) * 0.8)
train_textembedding, test_textembedding = x_train_seq[:train_size], x_train_seq[train_size:]
y_train_predict, y_test_predict = y_train[:train_size],y_train[train_size:]

In [405]:
#建立模型
#* 平坦層: 共有 3,200 = 32 * 100 個神經元, 共有 100 個 Token 數字, 每一個 Token 使用 32 維度來表示數字 1-2000.
#* 平坦層: 共有 250 個 Token 數字, 每一個 Token 使用 250 維度來表示數字 1.
#* 隱藏層: 共有 256 個神經元
#* 輸出層: 只有一個神經元. 1 表示正面; 0 表示負面.

model = Sequential()
#model.add(Embedding(output_dim=32,  input_dim=2000,  input_length=100))  
#model.add(Embedding(output_dim=250,  input_dim=11,  input_length=250))
#model.add(Dropout(0.2))
#model.add(Flatten())
model.add(Dense(units=256,activation='relu',input_dim=250 ))
model.add(Dropout(0.2))
model.add(Dense(units=1,activation='sigmoid' ))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 256)               64256     
_________________________________________________________________
dropout_43 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 257       
Total params: 64,513
Trainable params: 64,513
Non-trainable params: 0
_________________________________________________________________


In [406]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [407]:
train_history =model.fit(np.array(train_textembedding), np.array(y_train_predict),batch_size=5, epochs=5,verbose=2,validation_split=0.2)

Train on 550 samples, validate on 138 samples
Epoch 1/5
 - 1s - loss: 0.4535 - acc: 0.7927 - val_loss: 0.3849 - val_acc: 0.8478
Epoch 2/5
 - 0s - loss: 0.2939 - acc: 0.8727 - val_loss: 0.2957 - val_acc: 0.8841
Epoch 3/5
 - 0s - loss: 0.2212 - acc: 0.9145 - val_loss: 0.2841 - val_acc: 0.8551
Epoch 4/5
 - 0s - loss: 0.1789 - acc: 0.9455 - val_loss: 0.2761 - val_acc: 0.8551
Epoch 5/5
 - 0s - loss: 0.1563 - acc: 0.9455 - val_loss: 0.2388 - val_acc: 0.8986


In [408]:
%pylab inline
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

Populating the interactive namespace from numpy and matplotlib


In [409]:
show_train_history(train_history,'acc','val_acc')

In [410]:
show_train_history(train_history,'loss','val_loss')

In [411]:
#測試準確率
scores = model.evaluate(np.array(test_textembedding), np.array(y_test_predict), verbose=1)
scores[1]

173/173 [==============================] - 0s 29us/step


0.8786127167630058

In [412]:
#預測機率
probility=model.predict(np.array(test_textembedding))

In [384]:
predict=model.predict_classes(np.array(test_textembedding))

In [413]:
predict.shape

(173, 1)

In [414]:
predict_classes=predict.reshape(173)

In [415]:
SentimentDict={1:'屬於A',0:'不屬於A'}
def display_test_Sentiment(i):
    print(train_text[i])
    print('標籤label:',SentimentDict[y_train[i]],'預測結果:',SentimentDict[predict_classes[i]])

In [417]:
for i in range(0,20):
    display_test_Sentiment(i)
    print("\n")

夾
標籤label: 不屬於A 預測結果: 不屬於A


衣服
標籤label: 屬於A 預測結果: 不屬於A


裁縫
標籤label: 不屬於A 預測結果: 不屬於A


白
標籤label: 屬於A 預測結果: 不屬於A


民生
標籤label: 不屬於A 預測結果: 不屬於A


用品
標籤label: 屬於A 預測結果: 不屬於A


白色
標籤label: 屬於A 預測結果: 屬於A


白色
標籤label: 屬於A 預測結果: 屬於A


白色
標籤label: 屬於A 預測結果: 屬於A


裝
標籤label: 不屬於A 預測結果: 不屬於A


衣服
標籤label: 屬於A 預測結果: 屬於A


白色
標籤label: 屬於A 預測結果: 不屬於A


固體
標籤label: 屬於A 預測結果: 不屬於A


物資
標籤label: 屬於A 預測結果: 不屬於A


顏色
標籤label: 不屬於A 預測結果: 不屬於A


相似
標籤label: 不屬於A 預測結果: 屬於A


上班族
標籤label: 不屬於A 預測結果: 不屬於A


貨物
標籤label: 屬於A 預測結果: 屬於A


顏色
標籤label: 不屬於A 預測結果: 屬於A


髒
標籤label: 不屬於A 預測結果: 不屬於A


